In [1]:
import pandas as pd
import numpy as np
import time
from datetime import date
from matplotlib import pyplot as plt
from pylab import rcParams
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm_notebook
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
from tqdm import tqdm_notebook
from random import randint


df = pd.read_csv('Final_dataset(1).csv')


In [2]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ID,Time,Open,High,Low,Close,Quantity,...,Date,Support Levels Reached,Resistance Levels Reached,Chopiness Index,Fresh Milestone Indicator,Bull/Bear,Money Flow,Wild Swing Indicator,Initiation,Realized Spread
0,0,0,0,1,9:08:00,1759.50,1759.5,1759.50,1759.50,248,...,2018-01-01,Below S2,Crossed R3,0.820802,0.0,NaN,NaN,0.0,Seller Initited,-0.398237
1,1,1,1,2,9:16:00,1760.00,1760.0,1756.10,1758.20,840,...,2018-01-01,Below S2,Crossed R3,0.818419,0.0,Retracement,-1476888.00,0.0,Seller Initited,-0.068263
2,2,2,2,3,9:17:00,1758.15,1758.6,1757.85,1757.85,243,...,2018-01-01,Below S2,Crossed R3,0.885454,0.0,Retracement,-427157.55,0.0,Seller Initited,-0.085350
3,3,3,3,4,9:18:00,1756.85,1757.8,1756.00,1756.00,2166,...,2018-01-01,Below S2,Crossed R3,0.808434,0.0,Retracement,-3803496.00,0.0,Seller Initited,0.130937
4,4,4,4,5,9:19:00,1754.80,1755.0,1752.50,1752.50,1893,...,2018-01-01,Below S2,Crossed R3,0.742926,0.0,Retracement,-3317482.50,0.0,Seller Initited,0.512894


In [3]:
df['Date_Time']= pd.to_datetime(df['Date'] + ' ' + df['Time'])
new_df = df[['ID', 'Date_Time','Open','High','Low','Close','Quantity','Zone']].copy()
new_df.head()

,ID,Date_Time,Open,High,Low,Close,Quantity,Zone
0,1,2018-01-01 09:08:00,1759.50,1759.5,1759.50,1759.50,248,NaN
1,2,2018-01-01 09:16:00,1760.00,1760.0,1756.10,1758.20,840,Zone1
2,3,2018-01-01 09:17:00,1758.15,1758.6,1757.85,1757.85,243,Zone1
3,4,2018-01-01 09:18:00,1756.85,1757.8,1756.00,1756.00,2166,Zone1
4,5,2018-01-01 09:19:00,1754.80,1755.0,1752.50,1752.50,1893,Zone1


### Finding average volume traded over entire dataset(period can be changed later)

In [4]:
a=df['Date_Time'][134008]
vol=0
for i, row in df.iterrows():
    if(new_df['Date_Time'][i] < a):
        vol= new_df['Quantity'][i]+vol
    else:
        avg=vol/(i+1)
avg=vol/(i+1)

### Binning Volume Traded Ratio into L, ML, MH, H

In [5]:
ratio=[]
zones=[]
for i, row in new_df.iterrows():
    if(new_df['Date_Time'][i] <= a):
        ratio.append(new_df['Quantity'][i]/avg)
        zones.append(new_df['Zone'][i])
    else:
        break
zippedList =  list(zip(ratio,zones))
bins=pd.DataFrame(zippedList, columns=['Ratio', 'Zone'])
new_df['Volume Ratio']=bins['Ratio']
new_df['Volume Ratio bin']= pd.qcut(new_df['Volume Ratio'], 4, labels=['L', 'ML', 'MH', 'H'])
new_df.head()

,ID,Date_Time,Open,High,Low,Close,Quantity,Zone,Volume Ratio,Volume Ratio bin
0,1,2018-01-01 09:08:00,1759.50,1759.5,1759.50,1759.50,248,NaN,0.058469,L
1,2,2018-01-01 09:16:00,1760.00,1760.0,1756.10,1758.20,840,Zone1,0.198041,L
2,3,2018-01-01 09:17:00,1758.15,1758.6,1757.85,1757.85,243,Zone1,0.057290,L
3,4,2018-01-01 09:18:00,1756.85,1757.8,1756.00,1756.00,2166,Zone1,0.510662,MH
4,5,2018-01-01 09:19:00,1754.80,1755.0,1752.50,1752.50,1893,Zone1,0.446298,ML


### We take 100000 sequences of 10-30 events(trading points), and for each trading point we extract features like close gap, high gap etc(A set of basic features chosen for time being). These features will be our observable states. 

In [35]:
new_set = []
for row_set in range(0, 100000):
    if row_set%2000==0: print(row_set)
    row_quant = randint(10, 30)
    row_start = randint(0, len(new_df)-row_quant)
    market_subset = new_df.iloc[row_start:row_start+row_quant]
    
    Close_Time = max(market_subset['Date_Time'])
    if row_set%2000==0: print(Close_Time)
    
    # Close_Gap = (market_subset['Close'] - market_subset['Close'].shift(1)) / market_subset['Close'].shift(1)
    Close_Gap = market_subset['Close'].pct_change()
    High_Gap = market_subset['High'].pct_change()
    Low_Gap = market_subset['Low'].pct_change() 
    Date_Time=market_subset['Date_Time']
    Zone= market_subset['Zone']
    Volume_Ratio= market_subset['Volume Ratio']
    Volume_Ratio_bin= market_subset['Volume Ratio bin']
    Volume_Gap = market_subset['Quantity'].pct_change() 
    Daily_Change = (market_subset['Close'] - market_subset['Open']) / market_subset['Open']
    Vol_change_Direction = (market_subset['Quantity'].shift(-1) - market_subset['Quantity'])
    
    new_set.append(pd.DataFrame({'ID':[row_set]*len(market_subset),
                            'Close_Time':[Close_Time]*len(market_subset),
                           'Close_Gap':Close_Gap,
                           'High_Gap':High_Gap,
                            'Date': Date_Time,
                           'Low_Gap':Low_Gap,
                            'Zone':Zone,
                            'Volume_Ratio':Volume_Ratio,
                            'Volume_Ratio_bin':Volume_Ratio_bin,
                           'Volume_Gap':Volume_Gap,
                           'Daily_Change':Daily_Change,
                           'Vol_change_Direction':Vol_change_Direction}))

0
2018-03-27 10:00:00
2000
2019-01-29 09:55:00
4000
2019-03-28 11:09:00
6000
2018-11-13 15:48:00
8000
2018-05-11 11:29:00
10000
2018-12-06 12:00:00
12000
2019-05-03 09:59:00
14000
2018-04-10 14:01:00
16000
2018-09-24 10:42:00
18000
2019-05-08 11:11:00
20000
2019-04-18 10:26:00
22000
2018-12-27 11:22:00
24000
2018-08-31 14:25:00
26000
2018-05-15 13:41:00
28000
2018-03-22 14:08:00
30000
2018-10-25 15:49:00
32000
2019-02-12 12:00:00
34000
2019-03-01 12:39:00
36000
2019-04-22 11:05:00
38000
2018-11-28 10:04:00
40000
2018-07-03 12:56:00
42000
2018-07-04 12:56:00
44000
2019-01-03 11:53:00
46000
2018-05-15 13:22:00
48000
2018-05-04 10:03:00
50000
2019-01-09 09:56:00
52000
2019-01-25 13:07:00
54000
2018-12-14 11:30:00
56000
2018-10-04 11:09:00
58000
2018-12-19 14:58:00
60000
2019-05-08 10:49:00
62000
2019-02-26 11:47:00
64000
2019-03-07 14:41:00
66000
2019-05-06 12:37:00
68000
2018-06-15 16:00:00
70000
2018-09-21 09:18:00
72000
2018-06-19 15:15:00
74000
2018-03-09 10:27:00
76000
2018-06-01 14:

### The resulting dataframe consists of 100000's of sequences

In [36]:
new_set_df = pd.concat(new_set)
print(new_set_df.shape)
new_set_df = new_set_df.dropna(how='any') 
print(new_set_df.shape)
new_set_df.tail(10)

(2003538, 12)
(1756501, 12)


,ID,Close_Time,Close_Gap,High_Gap,Date,Low_Gap,Zone,Volume_Ratio,Volume_Ratio_bin,Volume_Gap,Daily_Change,Vol_change_Direction
92407,99999,2018-12-24 09:56:00,-0.000156,0.000292,2018-12-24 09:46:00,0.000449,Zone1,0.772594,MH,0.169105,-0.000156,2839.0
92408,99999,2018-12-24 09:56:00,-0.001890,-0.000584,2018-12-24 09:47:00,-0.001580,Zone1,1.441924,H,0.866341,-0.001754,-3300.0
92409,99999,2018-12-24 09:56:00,-0.000605,-0.001481,2018-12-24 09:48:00,-0.000098,Zone1,0.663907,MH,-0.539568,-0.000878,2056.0
92410,99999,2018-12-24 09:56:00,-0.000957,-0.000878,2018-12-24 09:49:00,-0.001114,Zone1,1.148635,H,0.730114,-0.000782,4771.0
92411,99999,2018-12-24 09:56:00,0.001251,0.000684,2018-12-24 09:50:00,-0.000391,Zone1,2.273458,H,0.979269,0.001251,-5978.0
92412,99999,2018-12-24 09:56:00,-0.001211,-0.000644,2018-12-24 09:51:00,0.000646,Zone1,0.864070,MH,-0.619932,-0.000801,1575.0
92413,99999,2018-12-24 09:56:00,0.002561,0.001602,2018-12-24 09:52:00,0.000332,Zone1,1.235396,H,0.429741,0.002346,-2275.0
92414,99999,2018-12-24 09:56:00,-0.001424,-0.000644,2018-12-24 09:53:00,0.000196,Zone1,0.699036,MH,-0.434160,-0.000781,-701.0
92415,99999,2018-12-24 09:56:00,-0.000410,-0.000781,2018-12-24 09:54:00,0.000000,Zone1,0.533766,MH,-0.236425,-0.000410,1077.0
92416,99999,2018-12-24 09:56:00,-0.000469,-0.000605,2018-12-24 09:55:00,-0.001134,Zone1,0.787683,MH,0.475707,-0.000274,-1503.0


### Now we bin the features in order to have discrete states for our Markov Model. We bin them into 3 categories- L,M,H. So instead of having continous values as hidden states we have dicrete varibles

In [37]:
# Close_Gap
new_set_df['Close_Gap_LMH'] = pd.qcut(new_set_df['Close_Gap'], 3, labels=["L", "M", "H"])

# High_Gap - not used in this example
new_set_df['High_Gap_LMH'] = pd.qcut(new_set_df['High_Gap'], 3, labels=["L", "M", "H"])

# Low_Gap - not used in this example
new_set_df['Low_Gap_LMH'] = pd.qcut(new_set_df['Low_Gap'], 3, labels=["L", "M", "H"])

# Volume_Gap
new_set_df['Volume_Gap_LMH'] = pd.qcut(new_set_df['Volume_Gap'], 3, labels=["L", "M", "H"])
 
# Daily_Change
new_set_df['Daily_Change_LMH'] = pd.qcut(new_set_df['Daily_Change'], 3, labels=["L", "M", "H"])

new_set_df=new_set_df.drop(columns=['Close_Gap', 'High_Gap', 'Low_Gap', 'Volume_Gap', 'Daily_Change'])

new_set_df.head()

,ID,Close_Time,Date,Zone,Volume_Ratio,Volume_Ratio_bin,Vol_change_Direction,Close_Gap_LMH,High_Gap_LMH,Low_Gap_LMH,Volume_Gap_LMH,Daily_Change_LMH
22235,0,2018-03-27 10:00:00,2018-03-27 09:40:00,Zone1,0.523864,MH,-630.0,H,H,H,L,H
22236,0,2018-03-27 10:00:00,2018-03-27 09:41:00,Zone1,0.375334,ML,5109.0,M,L,H,M,M
22237,0,2018-03-27 10:00:00,2018-03-27 09:42:00,Zone1,1.579845,H,1960.0,M,H,H,H,H
22238,0,2018-03-27 10:00:00,2018-03-27 09:43:00,Zone1,2.041939,H,-3594.0,L,L,L,M,L
22239,0,2018-03-27 10:00:00,2018-03-27 09:44:00,Zone1,1.194609,H,5722.0,H,M,M,L,H


### Now we concatenate our binned features, which will represent the sequence of observations. The Markov Model is expected to predict how a variable changes(in this case volume change) given this sequence of observations(Check last column of below dataframe for sequence)

In [38]:
event_df = new_set_df[["ID",
                         "Zone",
                         "Volume_Ratio",
                         "Volume_Ratio_bin",                         
                         "Close_Time", 
                         "Close_Gap_LMH", 
                         "Volume_Gap_LMH", 
                         "High_Gap_LMH",
                         "Low_Gap_LMH",
                         "Daily_Change_LMH", 
                         "Vol_change_Direction"]]


event_df['Event_Pattern'] = event_df['Close_Gap_LMH'].astype(str) + event_df['Volume_Gap_LMH'].astype(str) + event_df['High_Gap_LMH'].astype(str)+ event_df['Low_Gap_LMH'].astype(str)+ event_df['Daily_Change_LMH'].astype(str)
event_df.head()



,ID,Zone,Volume_Ratio,Volume_Ratio_bin,Close_Time,Close_Gap_LMH,Volume_Gap_LMH,High_Gap_LMH,Low_Gap_LMH,Daily_Change_LMH,Vol_change_Direction,Event_Pattern
22235,0,Zone1,0.523864,MH,2018-03-27 10:00:00,H,L,H,H,H,-630.0,HLHHH
22236,0,Zone1,0.375334,ML,2018-03-27 10:00:00,M,M,L,H,M,5109.0,MMLHM
22237,0,Zone1,1.579845,H,2018-03-27 10:00:00,M,H,H,H,H,1960.0,MHHHH
22238,0,Zone1,2.041939,H,2018-03-27 10:00:00,L,M,L,L,L,-3594.0,LMLLL
22239,0,Zone1,1.194609,H,2018-03-27 10:00:00,H,L,M,M,H,5722.0,HLMMH


In [50]:
# reduce the set
compressed_set = event_df.groupby(['ID','Close_Time'])['Event_Pattern'].apply(lambda x: "{%s}" % ', '.join(x)).reset_index()
print(compressed_set.shape)
compressed_set.head() 

(99844, 3)


,ID,Close_Time,Event_Pattern
0,0,2018-03-27 10:00:00,"{HLHHH, MMLHM, MHHHH, LMLLL, HLMMH, HHHMH, LLM..."
1,1,2018-05-31 14:38:00,"{MHLLL, LLLLL, HMMMH, HLHHH, HMHHH, LLHML, LHL..."
2,2,2018-03-23 13:13:00,"{MHHMM, HMHMH, LHMHL, HHHHH, LMHHL, HMHMH, LLL..."
3,3,2018-09-24 09:52:00,"{HLLHH, MMHHM, LLLLL, LHLLM, HLHHH, HMHHH, LHL..."
4,4,2018-06-20 12:26:00,"{HLHMH, MHMML, MLMMM, MHLLM, HMHHH, MMHHM, LML..."


In [51]:
#Alternate Target variable (change in volume direction)
compressed_outcomes = event_df.groupby(['ID', 'Close_Time'])['Vol_change_Direction'].mean()
compressed_outcomes = compressed_outcomes.to_frame().reset_index()
print(compressed_outcomes.shape)
compressed_outcomes.describe()

(99844, 3)


,ID,Vol_change_Direction
count,99844.000000,99844.000000
mean,49993.450112,-88.783510
std,28867.730336,957.781438
min,0.000000,-69760.000000
25%,24994.750000,-125.037037
50%,49991.500000,-8.833333
75%,74990.250000,71.175889
max,99999.000000,76787.375000


In [29]:
compressed_outcomes = event_df.groupby(['ID', 'Close_Time'])['Volume_Ratio'].mean()
compressed_outcomes = compressed_outcomes.to_frame().reset_index()
print(compressed_outcomes.shape)
compressed_outcomes.describe()

(99831, 3)


,ID,Volume_Ratio
count,99831.000000,99831.000000
mean,49996.007863,1.060654
std,28866.851147,1.422920
min,0.000000,0.014786
25%,25001.500000,0.359153
50%,49992.000000,0.644321
75%,74997.500000,1.212056
max,99999.000000,31.982719


In [52]:
compressed_set = pd.merge(compressed_set, compressed_outcomes, on= ['ID','Close_Time'], how='inner')
print(compressed_set.shape)
compressed_set.head()

(99844, 4)


,ID,Close_Time,Event_Pattern,Vol_change_Direction
0,0,2018-03-27 10:00:00,"{HLHHH, MMLHM, MHHHH, LMLLL, HLMMH, HHHMH, LLM...",562.600000
1,1,2018-05-31 14:38:00,"{MHLLL, LLLLL, HMMMH, HLHHH, HMHHH, LLHML, LHL...",-118.160000
2,2,2018-03-23 13:13:00,"{MHHMM, HMHMH, LHMHL, HHHHH, LMHHL, HMHMH, LLL...",22.583333
3,3,2018-09-24 09:52:00,"{HLLHH, MMHHM, LLLLL, LHLLM, HLHHH, HMHHH, LHL...",1310.625000
4,4,2018-06-20 12:26:00,"{HLHMH, MHMML, MLMMM, MHLLM, HMHHH, MMHHM, LML...",83.916667


### Our goal is to predict level of Volume Ratio, we bin it into 4 categories L,ML,MH,H(was not able to complete this task)

In [31]:
compressed_set['Volume Ratio bin']= pd.qcut(compressed_set['Volume_Ratio'], 4, labels=['L', 'ML', 'MH', 'H'])
from sklearn.preprocessing import LabelEncoder
number=LabelEncoder()
compressed_set['Volume Ratio bin']= number.fit_transform(compressed_set['Volume Ratio bin'].astype('str'))
compressed_set.head()

,ID,Close_Time,Event_Pattern,Volume_Ratio,Volume Ratio bin
0,0,2018-02-16 10:40:00,"{MHHMH, HHHHH, LHMLM, HLHHM, MLLMH, LHHLL, HLL...",0.264629,1
1,1,2019-05-29 15:18:00,"{MHHHM, HMMMH, MMHHM, LLLLL, MMLMH, MMMMM, MHH...",1.175198,2
2,2,2018-03-13 12:07:00,"{HLHMH, HLMHL, HMMLM, MHHHM, HLHHM, HMMMH, LHM...",0.392739,3
3,3,2019-02-12 12:16:00,"{HMHLH, LMMHL, LHLLL, LLLMM, HHHMH, HHHHH, LLM...",0.227764,1
4,4,2018-12-11 11:02:00,"{LHLLL, HLLHH, LHHLL, HLLMM, LHLLL, LMLML, MML...",1.031267,2


In [53]:
compressed_set['Event_Pattern'] = [''.join(e.split()).replace('{','')
                                   .replace('}','') for e in compressed_set['Event_Pattern'].values]


In [54]:
import datetime
compressed_set_validation = compressed_set[compressed_set['Close_Time'] >= datetime.datetime.now() 
                                           - datetime.timedelta(days=50)] # Sys.Date()-90 

print(compressed_set_validation.shape)
compressed_set = compressed_set[compressed_set['Close_Time'] < datetime.datetime.now() 
                                           - datetime.timedelta(days=50)]  
print(compressed_set.shape)

(5751, 4)
(94093, 4)


### Final sequence to predict change in volume traded direction

In [55]:
compressed_set.head()

,ID,Close_Time,Event_Pattern,Vol_change_Direction
0,0,2018-03-27 10:00:00,"HLHHH,MMLHM,MHHHH,LMLLL,HLMMH,HHHMH,LLMHL,LLMH...",562.600000
1,1,2018-05-31 14:38:00,"MHLLL,LLLLL,HMMMH,HLHHH,HMHHH,LLHML,LHLLL,LHLL...",-118.160000
2,2,2018-03-23 13:13:00,"MHHMM,HMHMH,LHMHL,HHHHH,LMHHL,HMHMH,LLLLM,LHLL...",22.583333
3,3,2018-09-24 09:52:00,"HLLHH,MMHHM,LLLLL,LHLLM,HLHHH,HMHHH,LHLLL,HHHLH",1310.625000
4,4,2018-06-20 12:26:00,"HLHMH,MHMML,MLMMM,MHLLM,HMHHH,MMHHM,LMLLL,LLLL...",83.916667


In [57]:
compressed_set['Vol_change_Direction'] = np.where((compressed_set['Vol_change_Direction'] > 0), 1, 0)
compressed_set_validation['Vol_change_Direction'] = np.where((compressed_set_validation['Vol_change_Direction'] > 0), 1, 0)
compressed_set.head()


,ID,Close_Time,Event_Pattern,Vol_change_Direction
0,0,2018-03-27 10:00:00,"HLHHH,MMLHM,MHHHH,LMLLL,HLMMH,HHHMH,LLMHL,LLMH...",1
1,1,2018-05-31 14:38:00,"MHLLL,LLLLL,HMMMH,HLHHH,HMHHH,LLHML,LHLLL,LHLL...",0
2,2,2018-03-23 13:13:00,"MHHMM,HMHMH,LHMHL,HHHHH,LMHHL,HMHMH,LLLLM,LHLL...",1
3,3,2018-09-24 09:52:00,"HLLHH,MMHHM,LLLLL,LHLLM,HLHHH,HMHHH,LHLLL,HHHLH",1
4,4,2018-06-20 12:26:00,"HLHMH,MHMML,MLMMM,MHLLM,HMHHH,MMHHM,LMLLL,LLLL...",1


### Creating 2 sets for +ve and -ve volume change. We find tranistion probabilties for both

In [58]:
compressed_set_pos = compressed_set[compressed_set['Vol_change_Direction']==1][['ID', 'Event_Pattern']]
print(compressed_set_pos.shape)
compressed_set_neg = compressed_set[compressed_set['Vol_change_Direction']==0][['ID', 'Event_Pattern']]
print(compressed_set_neg.shape)

(43329, 2)
(50764, 2)


In [59]:
flat_list = [item.split(',') for item in compressed_set['Event_Pattern'].values ]
unique_patterns = ','.join(str(r) for v in flat_list for r in v)
unique_patterns = list(set(unique_patterns.split(',')))
len(unique_patterns)

239

### I faced some trouble running the model to classify between 4 categories, so I changed my target variable to check Direction of Volume change in the next trading point.

In [60]:
def build_transition_grid(compressed_grid, unique_patterns):
    # build the markov transition grid

    patterns = []
    counts = []
    for from_event in unique_patterns:

        # how many times 
        for to_event in unique_patterns:
            pattern = from_event + ',' + to_event # MMM,MlM

            ids_matches = compressed_grid[compressed_grid['Event_Pattern'].str.contains(pattern)]
            found = 0
            if len(ids_matches) > 0:
                Event_Pattern = '---'.join(ids_matches['Event_Pattern'].values)
                found = Event_Pattern.count(pattern)
            patterns.append(pattern)
            counts.append(found)

    # create to/from grid
    grid_Df = pd.DataFrame({'pairs':patterns, 'counts': counts})

    grid_Df['x'], grid_Df['y'] = grid_Df['pairs'].str.split(',', 1).str
    grid_Df.head()

    grid_Df = grid_Df.pivot(index='x', columns='y', values='counts')

    grid_Df.columns= [col for col in grid_Df.columns]
    del grid_Df.index.name

    # replace all NaN with zeros
    grid_Df.fillna(0, inplace=True)
    grid_Df.head()

    #grid_Df.rowSums(transition_dataframe) 
    grid_Df = grid_Df / grid_Df.sum(1)
    return (grid_Df)

In [ ]:
grid_pos = build_transition_grid(compressed_set_pos, unique_patterns) 
grid_neg = build_transition_grid(compressed_set_neg, unique_patterns)